## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,66.47,30,53,1.39,broken clouds
1,1,Ligayan,PH,4.6533,119.4706,82.83,74,62,9.80,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,51.46,71,75,10.36,broken clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,-9.76,71,39,7.56,scattered clouds
4,4,Iqaluit,CA,63.7506,-68.5145,-9.31,76,100,0.00,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
while True:
    try:
        min_temp = float(input("The MINimum temperature(°F) for your trip? "))
        break
    except ValueError:
        print("Ooops! That was not a valid temperature. Please try again.")
while True:
    try:
        max_temp = float(input("The MAXimum temperature(°F) for your trip? "))
        break
    except ValueError:
        print("Ooops! That was not a valid temperature. Please try again.")
    

The MINimum temperature(°F) for your trip? 50
The MAXimum temperature(°F) for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,66.47,30,53,1.39,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,51.46,71,75,10.36,broken clouds
7,7,Souillac,MU,-20.5167,57.5167,73.78,83,40,5.75,scattered clouds
8,8,Bredasdorp,ZA,-34.5322,20.0403,65.28,88,1,6.91,clear sky
9,9,Carutapera,BR,-1.1950,-46.0200,73.27,96,100,5.12,overcast clouds
11,11,Busselton,AU,-33.6500,115.3333,67.05,60,0,12.35,clear sky
13,13,Maldonado,UY,-34.6667,-54.9167,64.22,68,21,5.08,few clouds
15,15,Paita,PE,-5.0892,-81.1144,71.35,84,45,11.97,scattered clouds
17,17,Rikitea,PF,-23.1203,-134.9692,78.73,75,84,17.45,broken clouds
20,20,Jackson,US,42.4165,-122.8345,62.02,30,0,0.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

City_ID                246
City                   246
Country                246
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
dtype: int64


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,66.47,broken clouds,-46.1927,168.8643,
2,Ushuaia,AR,51.46,broken clouds,-54.8000,-68.3000,
7,Souillac,MU,73.78,scattered clouds,-20.5167,57.5167,
8,Bredasdorp,ZA,65.28,clear sky,-34.5322,20.0403,
9,Carutapera,BR,73.27,overcast clouds,-1.1950,-46.0200,
11,Busselton,AU,67.05,clear sky,-33.6500,115.3333,
13,Maldonado,UY,64.22,few clouds,-34.6667,-54.9167,
15,Paita,PE,71.35,scattered clouds,-5.0892,-81.1144,
17,Rikitea,PF,78.73,broken clouds,-23.1203,-134.9692,
20,Jackson,US,62.02,clear sky,42.4165,-122.8345,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
#check for null values and empty strings
empty_hotels = hotel_df.loc[hotel_df["Hotel Name"]== ""]
print(len(empty_hotels))

13


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!= ""].dropna()
clean_hotel_df.count()

City                   233
Country                233
Max Temp               233
Current Description    233
Lat                    233
Lng                    233
Hotel Name             233
dtype: int64

In [11]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,66.47,broken clouds,-46.1927,168.8643,Ellie's Villa
2,Ushuaia,AR,51.46,broken clouds,-54.8000,-68.3000,Albatross Hotel
7,Souillac,MU,73.78,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
8,Bredasdorp,ZA,65.28,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
9,Carutapera,BR,73.27,overcast clouds,-1.1950,-46.0200,Lidera


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))